In [26]:
import os
import sys
sys.path.append('C:\\Users\\xpy\\work\\strategies2py\\AprilQuant')
import pandas as pd
import matplotlib.pyplot as plt
import pymongo
import numpy as np

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["AprilQuant"]
col = db['chandelier']

In [2]:
def comparison_plot(curves, title, legends):
    fig=plt.figure(figsize=(16, 9))
    plt.title(title)
    for curve in curves:
        plt.plot(curve)
    plt.legend(legends)

In [9]:
length = 100
ema_length = 40
cursor = db.chandelier.find({'params.enter_signal.length':40,
                             'params.enter_signal.ema_length':100})

In [10]:
documents = list(cursor)

In [21]:
cr0 = pd.DataFrame(documents[0]['bt_cum_ret']).set_index('index')
cr1 = pd.DataFrame(documents[31]['bt_cum_ret']).set_index('index') 

In [30]:
cr1

,cum_ret
index,
2018-10-25 00:00:00.000,1.000000
2018-10-25 23:59:59.999,1.000000
2018-10-26 00:00:00.000,1.000000
2018-10-26 23:59:59.999,1.000000
2018-10-29 00:00:00.000,1.000000
...,...
2021-04-26 23:59:59.999,0.996666
2021-04-27 00:00:00.000,0.996666
2021-04-27 23:59:59.999,0.996666


In [31]:
pd.concat([cr0, cr1], axis=1)

,cum_ret,cum_ret
index,,
2010-07-27 00:00:00.000,1.0,NaN
2010-07-27 23:59:59.999,1.0,NaN
2010-07-28 00:00:00.000,1.0,NaN
2010-07-28 23:59:59.999,1.0,NaN
2010-07-29 00:00:00.000,1.0,NaN
...,...,...
2021-04-26 23:59:59.999,NaN,0.996666
2021-04-27 00:00:00.000,NaN,0.996666
2021-04-27 23:59:59.999,NaN,0.996666


In [ ]:
def cal_avg_cum_ret(cum_rets, time_range):
    """
    计算平均累计收益率。
    cum_rets:defaultdict(list)
    """
    c_and_rs = [cal_cost_and_ret(cum_ret)
                for cum_ret in cum_rets]
    
    expanded_c_and_rs = [expand(c_and_r, time_range).fillna(0) for c_and_r in c_and_rs]
    sum_c_and_r = sum(expanded_c_and_rs)
    sum_c_and_r['avg_ret'] = (sum_c_and_r['ret'] / sum_c_and_r['cost']).fillna(0)
    sum_c_and_r['avg_cum_ret'] = (1 + sum_c_and_r['avg_ret']).cumprod()
    return sum_c_and_r['avg_cum_ret']

In [4]:
def get_curve_from_document(curve_name):
    df
    
    

In [6]:
document0 = next(cursor)


In [7]:
bt_cum_ret = pd.DataFrame(data=document['bt_cum_ret']).set_index('index')
pd_cum_ret = pd.DataFrame(data=document['pd_cum_ret']).set_index('index')

NameError: name 'document' is not defined

In [ ]:
comparison_plot([bt_cum_ret, pd_cum_ret], '', ['backtrader', 'pandas'])    

In [ ]:
bt_cum_ret.plot()
bt_cum_ret.plot()